In [26]:
import json
import re
import time
from typing import Any, Dict, List, Optional, Union
from urllib.parse import quote_plus

import pandas as pd
import requests
from tqdm.notebook import tqdm
from retry.api import retry_call
import os
os.chdir('/mnt/e/erya/socialmedia_ads/IG')

sf_api_key = "INhGdYFecrKghvYft3BTGPQxYgP27oobSIaM74zvehIWJWuAEKhkXVE3dVpWgaFjBx90YDwaVxPFyRyUTO"
usernames = ['gucci','thenorthface','oldnavy']

def parse_posts(response_json: Dict[str, Any]) -> List[Dict[str, Any]]:
    top_level_key = "graphql" if "graphql" in response_json else "data"
    user_data = response_json[top_level_key].get("user", {})
    post_edges = user_data.get("edge_owner_to_timeline_media", {}).get("edges", [])
    posts = []
    for node in post_edges:
        post_json = node.get("node", {})
        shortcode = post_json.get("shortcode")
        image_url = post_json.get("display_url")
        caption_edges = post_json.get("edge_media_to_caption", {}).get("edges", [])
        description = caption_edges[0].get("node", {}).get("text") if len(caption_edges) > 0 else None
        n_comments = post_json.get("edge_media_to_comment", {}).get("count")
        likes_key = "edge_liked_by" if "edge_liked_by" in post_json else "edge_media_preview_like"
        n_likes = post_json.get(likes_key, {}).get("count")
        timestamp = post_json.get("taken_at_timestamp")
        posts.append({
            "shortcode": shortcode,
            "image_url": image_url,
            "description": description,
            "n_comments": n_comments,
            "n_likes": n_likes,
            "timestamp": timestamp,
        })
    return post_edges

def parse_page_info(response_json: Dict[str, Any]) -> Dict[str, Union[Optional[bool], Optional[str]]]:
    top_level_key = "graphql" if "graphql" in response_json else "data"
    user_data = response_json[top_level_key].get("user", {})
    page_info = user_data.get("edge_owner_to_timeline_media", {}).get("page_info", {})
    return page_info

def download_media(url, media_type, post_id, short_code, username):
    save_path = '/mnt/e/erya/socialmedia_ads/IG/media/{}/{}'.format(username,post_id)
    if media_type == 'GraphImage':
        img_data = requests.get(url).content
        with open(save_path+'/{}_{}.jpg'.format(post_id, short_code), 'wb') as handler:
            handler.write(img_data)
    elif media_type == 'video':
        video_data = requests.get(url).content
        with open(save_path+'/{}_{}.mp4'.format(post_id, short_code), 'wb') as handler:
            handler.write(video_data)
    else:
        for i in url:
            if ".jpg" in i:
                img_data = requests.get(i).content
                with open(save_path+'/{}_{}_{}.jpg'.format(post_id,short_code,url.index(i)), 'wb') as handler:
                    handler.write(img_data)
            else:
                video_data = requests.get(i).content
                with open(save_path+'/{}_{}_{}.mp4'.format(post_id,short_code, url.index(i)), 'wb') as handler:
                    handler.write(video_data)

In [27]:
username = 'gucci'
print(username)
params = {
        "api_key": sf_api_key,
        "url": f"https://i.instagram.com/api/v1/users/web_profile_info/?username={username}",
        "headers": json.dumps({"x-ig-app-id": "936619743392459"}),
    }

def request_json(url, params) -> Dict[str, Any]:
    response = requests.get(url, params=params, timeout=95)
    response.raise_for_status()
    return response.json()

response_json = request_json(url="https://scraping.narf.ai/api/v1/", params=params)

user_id = response_json.get("data", {}).get("user", {}).get("id")
if not user_id:
    print(f"User {username} not found.")
    #return []
# parse the first batch of posts from user profile response
posts = parse_posts(response_json=response_json)
page_info = parse_page_info(response_json=response_json)
# get next page cursor
end_cursor = page_info.get("end_cursor")

gucci


In [6]:
posts

[{'node': {'__typename': 'GraphSidecar',
   'id': '3350453676508503391',
   'shortcode': 'C5_M94IsN1f',
   'dimensions': {'height': 1350, 'width': 1080},
   'display_url': 'https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.29350-15/439264111_937943764449946_11161142843437341_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=joeFyebMxaoAb5PkhDI&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfCdGfmCcUYrhOCgv56tLIgNjoQculsdvpjx5V8i9InCSw&oe=662A25A8&_nc_sid=8b3546',
   'edge_media_to_tagged_user': {'edges': [{'node': {'user': {'full_name': 'Cara Delevingne',
        'followed_by_viewer': False,
        'id': '3255807',
        'is_verified': True,
        'profile_pic_url': 'https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.2885-19/425472411_196872283483505_816557811738084692_n.jpg?stp=dst-jpg_s150x150&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=RlU7uwJ38v0Ab6Jrnh_&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfAiS_t1vxDBWwNimjFEAwSfsjP0WqMde9OHKI7AvYVs-g&oe=662A253A&_nc_s

In [28]:
if os.path.exists('meta/{}'.format(username)):
    pass
else:
    os.mkdir('meta/{}'.format(username))
if os.path.exists('media/{}'.format(username)):
    pass
else:
    os.mkdir('media/{}'.format(username))
for i in posts:
    print(posts.index(i))
    post_response = i['node']
    post_id = i['node']['id']
    short_code = i['node']['shortcode']
    media_type = i['node']['__typename']
    if os.path.exists('meta/{}/{}'.format(username,post_id)):
        pass
    else:
        os.mkdir('/mnt/e/erya/socialmedia_ads/IG/meta/{}/{}'.format(username,post_id))
        with open('/mnt/e/erya/socialmedia_ads/IG/meta/{}/{}/{}_{}.json'.format(username,post_id,post_id,short_code), 'w') as f:
            json.dump(i, f)
        print("Successfull saved")
    if os.path.exists('media/{}/{}'.format(username,post_id)):
        pass
    else:
        os.mkdir('media/{}/{}'.format(username,post_id)) 
        print('successfull created')
        if media_type == 'GraphSidecar':
            print('Multiple media')
            multimedia = post_response.get("edge_sidecar_to_children", {}).get("edges", [])
            num_media = len(multimedia)
            media_id = [i.get("node", {}).get('id') for i in multimedia]
            media_url = [i.get("node", {}).get('display_url') if i.get("node", {}).get("__typename") == "GraphImage" else i.get("node", {}).get('video_url') for i in multimedia]
            print(media_url)
            download_media(media_url, media_type,post_id, short_code, username)
            print('successfully downloaded')
            #n_view = [-1 if i.get("node", {}).get("__typename") == "GraphImage" else i.get("node", {}).get('video_view_count') for i in multimedia]
        elif media_type == "GraphImage":
            num_media = 1
            media_id = post_id
            media_url = post_response.get("display_url")
            download_media(media_url, media_type,post_id, short_code, username)
            #n_view = -1
        else:
            num_media = 1
            media_id = post_id
            media_url = post_response.get("video_url")
            #print(media_url)
            download_media(media_url, "video",post_id, short_code, username) 

0
1
2
3
successfull created


KeyboardInterrupt: 

In [21]:
post_response = posts[2]['node']
multimedia = post_response.get("edge_sidecar_to_children", {}).get("edges", [])
media_url = [i.get("node", {}).get('display_url') if i.get("node", {}).get("__typename") == "GraphImage" else i.get("node", {}).get('video_url') for i in multimedia]

In [22]:
len(media_url)

9

In [14]:
for i in posts:
    post_response = i['node']
    post_id = i['node']['id']
    short_code = i['node']['shortcode']
    media_type = i['node']['__typename']
    if os.path.exists('meta/{}/{}'.format(username,post_id)):
        pass
    else:
        os.mkdir('/mnt/e/erya/socialmedia_ads/IG/meta/{}/{}'.format(username,post_id))
        with open('/mnt/e/erya/socialmedia_ads/IG/meta/{}/{}/{}_{}.json'.format(username,post_id,post_id,short_code), 'w') as f:
            json.dump(i, f)
        print("Successfull saved")

Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved
Successfull saved


In [29]:
import urllib

opener = urllib.request.build_opener()
opener.addheaders = [("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:67.0) Gecko/20100101 Firefox/67.0")]
urllib.request.install_opener(opener)

In [30]:
import urllib3

def download_image(url, save_as):
    http = urllib3.PoolManager()
    response = http.request('GET', url)
    with open(save_as, 'wb') as file:
        file.write(response.data)

In [23]:
media_url

['https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.29350-15/438997522_783215470428397_4838710044579457554_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=EHMRecGVwBMAb5-vooB&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfDuhiQBO6deI6_r16NpHtP_UhtHbvyoa18AjmYfIfvJ0w&oe=662A2D16&_nc_sid=8b3546',
 'https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.29350-15/438987628_405007358986600_4105242626181198805_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=104&_nc_ohc=B0sUUMh9dGQAb6Aqox9&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfDKsGP4icPtaCKz7Mabp0QKp4wV81yA0TbjUjMOAdPkKA&oe=662A33B1&_nc_sid=8b3546',
 'https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.29350-15/439017519_1822020774966294_6996387975734448910_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=106&_nc_ohc=NQmIV836_9QAb6RohGx&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfAuQRQ5uvPwakiAXy9SzloH054ZkcuGNQsESMk50nJlKw&oe=662A4ABD&_nc_sid=8b3546',
 'https://instagram.fwaw7-1.fna.fbc

In [ ]:
for url in media_url:
    print(url)
    download_image(url, 'media/{}/{}'.format(username,post_id))

In [27]:
os.chdir('/mnt/e/erya/socialmedia_ads/IG/media/gucci/')

In [37]:
for url in media_url:
    print(url)
    #save_path = '/mnt/e/erya/socialmedia_ads/IG/media/{}/{}'.format(username,post_id)
    save_path ='/home/research/code/socialmedia_ads/{}_{}.jpg'.format(username, media_url.index(url))
    download_image(url, save_path)

https://scontent-waw2-1.cdninstagram.com/v/t51.29350-15/439235935_1171936010468916_3276406953997665676_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-waw2-1.cdninstagram.com&_nc_cat=100&_nc_ohc=v9ZeZA4pxhQAb60m0vl&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfBF4gkfDDC3S-Q3-E_-7wa0brXvX4nawsc8AFO3Pksl6g&oe=6629EAEA&_nc_sid=8b3546
https://scontent-waw2-1.cdninstagram.com/v/t51.29350-15/439012008_964278445092103_7449300640782622027_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-waw2-1.cdninstagram.com&_nc_cat=107&_nc_ohc=NkHeZ_ggKgcAb7OrWw8&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfA_g7XxhCk6ePQrcID3ayCjk9PFkBPtrafWwJhprBgY0w&oe=6629F965&_nc_sid=8b3546


KeyboardInterrupt: 

In [32]:
save_path ='/home/research/code/socialmedia_ads'
img_data = requests.get(media_url[2], stream=True).content
with open(save_path+'/{}_{}_{}.jpg'.format(post_id,short_code,1), 'wb') as handler:
    handler.write(img_data)

In [25]:
header = {'User-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'}
for url in media_url:
    print(url)
    #save_path = '/mnt/e/erya/socialmedia_ads/IG/media/{}/{}'.format(username,post_id)
    save_path ='/home/research/code/socialmedia_ads'
    img_data = requests.get(url, headers=header).content
    with open(save_path+'/{}_{}_{}.jpg'.format(post_id,short_code,media_url.index(url)), 'wb') as handler:
        handler.write(img_data)

https://instagram.fwaw7-1.fna.fbcdn.net/v/t51.29350-15/438997522_783215470428397_4838710044579457554_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=instagram.fwaw7-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=EHMRecGVwBMAb5-vooB&edm=AOQ1c0wBAAAA&ccb=7-5&oh=00_AfDuhiQBO6deI6_r16NpHtP_UhtHbvyoa18AjmYfIfvJ0w&oe=662A2D16&_nc_sid=8b3546


KeyboardInterrupt: 

In [8]:
save_path+'/{}_{}_{}.jpg'.format(post_id,short_code,url.index(i))

TypeError: must be str, not dict